# Setup

In [2]:
from utils import esg
from utils.ownership import parse_ownership_data
from utils.regressors import parse_regressors

In [3]:
import seaborn as sns

sns.set(style="white",
        palette="pastel",
        color_codes=True,
        rc={'figure.figsize': (12, 9) })

In [4]:
import pandas as pd
import statsmodels.api as sm

# Importing data

In [5]:
import dotenv, os

dotenv.load_dotenv()

data_path = os.path.join("..", os.environ.get("DATA_PATH"))

esg_path = os.path.join(data_path, "ESG.xlsx")
sheet_name = os.environ.get("SHEET_NAME")

own_path = os.path.join(data_path, "ownership.csv")
reg_path = os.path.join(data_path, "controls.csv")

saliency_path = os.path.join(data_path, "saliency.csv")

## Covariates

In [6]:
raw = pd.read_csv(reg_path)
covariates = parse_regressors(raw)

## ESG data

In [7]:
raw = pd.read_excel(esg_path, sheet_name = sheet_name)
get_esg = esg.yearly_esg_maker(raw)

## Ownership data

In [8]:
ownership = parse_ownership_data(pd.read_csv(own_path))
investors = ownership.columns[-7:]

In [9]:
ticker_to_name = dict(zip(ownership.TargetFirms, ownership.Tickers))

## Saliency data

In [10]:
saliency_df = pd.read_csv(saliency_path, index_col = [0, 1])

# Regression

In [12]:
saliency_df.loc["ANNUAL", "s-index"]

blackrock    7.477605e-08
dws          0.000000e+00
generali     1.498465e-05
jpmam        1.091731e-04
nn           4.881136e-04
apg          0.000000e+00
nbim         0.000000e+00
Name: (ANNUAL, s-index), dtype: float64

In [13]:
def makeregressors(saliency_df, ownership, covariates, index):

    reports = set([i[0] for i in saliency_df.index])
    
    companies = ownership.columns[-7:]
    X = ownership[companies]
    X.columns = [f"{company} ownership".lower() for company in companies]
    X.index = ownership['Tickers']

    for report in reports:
        S = saliency_df.loc[report, index]

        for company in S.index:
            column_name = f"{company} {report.lower()}"
            X.loc[:, f"{column_name} interaction"] = X[f"{company} ownership"] * S[company] 
    
    X = sm.add_constant(X, prepend = False, has_constant='add')

    common_indices = set(X.index).intersection(covariates.index)

    X = pd.concat([X.loc[common_indices], covariates.loc[common_indices]], axis = 1)

    return X

In [14]:
X = makeregressors(saliency_df, ownership, covariates, "s-index").dropna()

/Users/andreatitton/economics/nlp-financial-env/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [17]:
y = get_esg(2020)["ESG Combined Score"] / 100
y.index = [ticker_to_name.get(f, f) for f in  y.index]
common_indices = set(y.index).intersection(set(X.index))

y = y.loc[common_indices]
X = X.loc[common_indices]

In [18]:
model = sm.RLM(y, X, M=sm.robust.norms.HuberT())

In [31]:
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:     ESG Combined Score   No. Observations:                 2090
Model:                            RLM   Df Residuals:                     2064
Method:                          IRLS   Df Model:                           25
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 26 Nov 2021                                         
Time:                        11:57:47                                         
No. Iterations:                    50                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
nn ownership                                 1.5220      1.760      0.865      0.387      -1.927       4.971
blackrock ownership                         -0.4433      0.103     -4.303      0.000      -0.645      -0.241
nbim ownership                               5.8036      0.340     17.064      0.000       5.137       6.470
generali ownership                         220.0968     83.531      2.635      0.008      56.379     383.814
jpmam ownership                              0.5431      0.590      0.921      0.357      -0.613       1.699
apg ownership                                1.2069      3.874      0.312      0.755      -6.386       8.799
dws ownership                                3.6925      1.235      2.990      0.003       1.272       6.113
blackrock ri_report interaction             -0.0018      0.000     -4.300      0.000      -0.003      -0.001
dws ri_report interaction                    0.0160      0.005      2.991      0.003       0.006       0.027
generali ri_report interaction               1.9799      0.751      2.635      0.008       0.507       3.453
jpmam ri_report interaction                  0.0013      0.001      0.922      0.356      -0.001       0.004
nn ri_report interaction                     0.0019      0.002      0.868      0.385      -0.002       0.006
apg ri_report interaction                 6.709e-09   1.95e-09      3.447      0.001    2.89e-09    1.05e-08
nbim ri_report interaction                1.123e-10   3.21e-10      0.350      0.727   -5.17e-10    7.42e-10
blackrock voting_policy interaction      -5.851e-05   1.36e-05     -4.304      0.000   -8.52e-05   -3.19e-05
dws voting_policy interaction                0.0007      0.000      2.990      0.003       0.000       0.001
generali voting_policy interaction           0.5520      0.209      2.635      0.008       0.141       0.963
jpmam voting_policy interaction           5.476e-06   5.95e-06      0.921      0.357   -6.18e-06    1.71e-05
nn voting_policy interaction                 0.0004      0.000      0.865      0.387      -0.000       0.001
apg voting_policy interaction                0.0013      0.004      0.312      0.755      -0.007       0.010
nbim voting_policy interaction               0.0025      0.000     17.064      0.000       0.002       0.003
blackrock annual interaction             -3.315e-08    7.7e-09     -4.304      0.000   -4.83e-08   -1.81e-08
dws annual interaction                   -2.809e-14   7.83e-16    -35.850      0.000   -2.96e-14   -2.66e-14
generali annual interaction                  0.0033      0.001      2.635      0.008       0.001       0.006
jpmam annual interaction                  5.929e-05   6.44e-05      0.921      0.357   -6.69e-05       0.000
nn annual interaction          